# Extract Parameter from File Names
* Each file has:
    * <font color=blue>id</font> - the fish's unique id
    * <font color=green>test</font> - Hypoxia or Normoxia
    * <font color=cyan>date</font> - the date the data was collected
* for example:
    * files/CTswim Hyperoxia Cleaned/<font color=blue>411980</font>-CTswim-<font color=green>Hyperoxia</font>-Cleaned-<font color=cyan>29-09-2023</font>.csv




In [65]:
import pandas as pd
import os
import re
from datetime import datetime

directory = "./files"
file_list = os.listdir(directory)


## Use a 'regular expression' to extract the parameters

In [66]:
filename = "([0-9]+)-CTswim-(Hyperoxia|Normoxia)-Cleaned-([0-9_-]*).csv"
test = "541042-CTswim-Hyperoxia-Cleaned-25-08-2023.csv"
match = re.compile(filename).search(test)
match.groups()

('541042', 'Hyperoxia', '25-08-2023')

## List the files, and extract the parameters

In [67]:
matched_files = []

def flatten(xss):
    return [x for xs in xss for x in xs]

for root, subdirs, files in os.walk(directory):
    print(f"{root}")
    matched_files.append([({'file':root+'/'+f, 'id':match.group(1), 'test':match.group(2), 'date':match.group(3)}) for f in files
         if (match := re.compile(filename).search(f))])

file_list = flatten(matched_files)
file_list
        

./files
./files/CTswim Hyperoxia Cleaned
./files/CTswim Hyperoxia Cleaned/slopes_
./files/CTswim Normoxia Cleaned
./files/CTswim Normoxia Cleaned/Slopes


[{'file': './files/CTswim Hyperoxia Cleaned/541042-CTswim-Hyperoxia-Cleaned-25-08-2023.csv',
  'id': '541042',
  'test': 'Hyperoxia',
  'date': '25-08-2023'},
 {'file': './files/CTswim Hyperoxia Cleaned/412042-CTswim-Hyperoxia-Cleaned-21-09-2023.csv',
  'id': '412042',
  'test': 'Hyperoxia',
  'date': '21-09-2023'},
 {'file': './files/CTswim Hyperoxia Cleaned/412017-CTswim-Hyperoxia-Cleaned-02-12-2023.csv',
  'id': '412017',
  'test': 'Hyperoxia',
  'date': '02-12-2023'},
 {'file': './files/CTswim Hyperoxia Cleaned/541021-CTswim-Hyperoxia-Cleaned-03-12-2023.csv',
  'id': '541021',
  'test': 'Hyperoxia',
  'date': '03-12-2023'},
 {'file': './files/CTswim Hyperoxia Cleaned/412054-CTswim-Hyperoxia-Cleaned-11-09-2023.csv',
  'id': '412054',
  'test': 'Hyperoxia',
  'date': '11-09-2023'},
 {'file': './files/CTswim Hyperoxia Cleaned/412030-CTswim-Hyperoxia-Cleaned-03-12-2023.csv',
  'id': '412030',
  'test': 'Hyperoxia',
  'date': '03-12-2023'},
 {'file': './files/CTswim Hyperoxia Cleaned/41

## For each file, read the CSV, and write Excel

In [68]:
all_reports = []
total_rows = 0
for i, f in enumerate(file_list):
    c = pd.read_csv(f['file'])
    print(f"read {f['file']} for fish {f['id']} and test {f['test']} with shape {c.shape}")
    c['id']=f['id']
    c['test'] = f['test']
    c['date'] = f['date']
    c['average'] = (c['ch1']+c['ch2'])/2
    all_reports.append(c)
    total_rows+=c.shape[0]

print(f"total rows {total_rows}")



read ./files/CTswim Hyperoxia Cleaned/541042-CTswim-Hyperoxia-Cleaned-25-08-2023.csv for fish 541042 and test Hyperoxia with shape (16993, 5)
read ./files/CTswim Hyperoxia Cleaned/412042-CTswim-Hyperoxia-Cleaned-21-09-2023.csv for fish 412042 and test Hyperoxia with shape (13542, 5)
read ./files/CTswim Hyperoxia Cleaned/412017-CTswim-Hyperoxia-Cleaned-02-12-2023.csv for fish 412017 and test Hyperoxia with shape (14902, 5)
read ./files/CTswim Hyperoxia Cleaned/541021-CTswim-Hyperoxia-Cleaned-03-12-2023.csv for fish 541021 and test Hyperoxia with shape (10771, 5)
read ./files/CTswim Hyperoxia Cleaned/412054-CTswim-Hyperoxia-Cleaned-11-09-2023.csv for fish 412054 and test Hyperoxia with shape (12424, 5)
read ./files/CTswim Hyperoxia Cleaned/412030-CTswim-Hyperoxia-Cleaned-03-12-2023.csv for fish 412030 and test Hyperoxia with shape (15382, 5)
read ./files/CTswim Hyperoxia Cleaned/412056-CTswim-Hyperoxia-Cleaned-04-12-2023.csv for fish 412056 and test Hyperoxia with shape (14892, 5)
read .

In [69]:
all_reports[0].head()

,date,time,ch1,ch2,temp,id,test,average
0,25-08-2023,10:24:12,8.976,8.968,18.994,541042,Hyperoxia,8.9720
1,25-08-2023,10:24:13,8.970,8.983,18.992,541042,Hyperoxia,8.9765
2,25-08-2023,10:24:14,8.975,8.964,18.992,541042,Hyperoxia,8.9695
3,25-08-2023,10:24:15,8.983,8.970,18.992,541042,Hyperoxia,8.9765
4,25-08-2023,10:24:16,8.970,8.962,18.995,541042,Hyperoxia,8.9660


In [70]:
with pd.ExcelWriter('./files/merged_readings.xlsx', engine='xlsxwriter') as writer: 
    pd.concat(all_reports).to_excel(writer)